## Data Science Academy Classification Hackathon 


South Africa is a multicultural society that is characterised by its rich linguistic diversity. Language is an indispensable tool that can be used to deepen democracy and also contribute to the social, cultural, intellectual, economic and political life of the South African society.

The country is multilingual with 11 official languages, each of which is guaranteed equal status. Most South Africans are multilingual and able to speak at least two or more of the official languages.
From South African Government



### All about the data

The dataset used for this challenge is the NCHLT Text Corpora collected by the South African Department of Arts and Culture & Centre for Text Technology (CTexT, North-West University, South Africa). The training set was improved through additional cleaning done by Praekelt.

The data is in the form Language ID, Text. The text is in various states of cleanliness. Some NLP techniques will be necessary to clean up the data.

### Let's do some importing 

In [1]:
#importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
import re
import string
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk import word_tokenize, pos_tag, pos_tag_sents
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


### Let's do some data loading

In [2]:

data = pd.read_csv(r"C:\Users\manyi\Downloads\sample_submission.csv")
train = pd.read_csv(r"C:\Users\manyi\Downloads\train_set.csv\train_set.csv")
test = pd.read_csv(r"C:\Users\manyi\Downloads\test_set.csv")


### Exploring the data a little

In [3]:
sns.countplot(train.lang_id)

C:\Users\manyi\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


<AxesSubplot:xlabel='lang_id', ylabel='count'>

In [4]:
train.info

<bound method DataFrame.info of       lang_id                                               text
0         xho  umgaqo-siseko wenza amalungiselelo kumaziko ax...
1         xho  i-dha iya kuba nobulumko bokubeka umsebenzi na...
2         eng  the province of kwazulu-natal department of tr...
3         nso  o netefatša gore o ba file dilo ka moka tše le...
4         ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...
...       ...                                                ...
32995     tsn  popo ya dipolateforomo tse ke go tlisa boetele...
32996     sot  modise mosadi na o ntse o sa utlwe hore thaban...
32997     eng  closing date for the submission of completed t...
32998     xho  nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999     sot  mafapha a mang le ona a lokela ho etsa ditlale...

[33000 rows x 2 columns]>

In [5]:
train.head()

lang_id                                               text
0     xho  umgaqo-siseko wenza amalungiselelo kumaziko ax...
1     xho  i-dha iya kuba nobulumko bokubeka umsebenzi na...
2     eng  the province of kwazulu-natal department of tr...
3     nso  o netefatša gore o ba file dilo ka moka tše le...
4     ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...

In [6]:
data_lang = train.copy()

In [7]:
def hashtag_extract(tweet):  
    
  
    hashtags = []

    for i in tweet:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    hashtags = sum(hashtags, [])
    frequency = nltk.FreqDist(hashtags)

    hashtag_data = pd.DataFrame({'hashtag': list(frequency.keys()),
                           'count': list(frequency.values())})
    hashtag_data = hashtag_data.nlargest(15, columns="count")

    return hashtag_data

In [8]:
#remove all url/websites
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+' #url regualr expressions
subs_url = r'url-web' # replace each url with 'url-web'
data_lang['text'] = data_lang['text'].replace(to_replace = pattern_url, value = subs_url, regex = True)
# make all lower case 
data_lang['text'] = data_lang['text'].str.lower()

#Removing RT ftom tweets
data_lang['text'] = data_lang['text'].str.strip('rt ')

# Remove @ mentions
pattern = r"@[\w]+" # pattern to remove
sub = r'' # what to replace it with
data_lang['text'] = data_lang['text'].replace(to_replace = pattern, value = sub, regex = True) #replace

In [9]:
def lookup_dict(text, dictionary):
    
    for word in text.split(): 
        if word.lower() in dictionary: 
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()]) 
    return text

In [10]:
#remove puntuation
data_lang['text'] = data_lang['text'].apply(lambda x: ''.join([l for l in x if l not in string.punctuation]))
data_lang

lang_id                                               text
0         xho  umgaqosiseko wenza amalungiselelo kumaziko axh...
1         xho  idha iya kuba nobulumko bokubeka umsebenzi nap...
2         eng  he province of kwazulunatal department of tran...
3         nso  o netefatša gore o ba file dilo ka moka tše le...
4         ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...
...       ...                                                ...
32995     tsn  popo ya dipolateforomo tse ke go tlisa boetele...
32996     sot  modise mosadi na o ntse o sa utlwe hore thaban...
32997     eng  closing date for the submission of completed t...
32998     xho  nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999     sot  mafapha a mang le ona a lokela ho etsa ditlale...

[33000 rows x 2 columns]

In [11]:
#tokenizing the tweets
tokeniser = TreebankWordTokenizer()
data_lang['tokenized'] = data_lang['text'].apply(tokeniser.tokenize)

In [12]:
data_lang.head()

lang_id                                               text  \
0     xho  umgaqosiseko wenza amalungiselelo kumaziko axh...   
1     xho  idha iya kuba nobulumko bokubeka umsebenzi nap...   
2     eng  he province of kwazulunatal department of tran...   
3     nso  o netefatša gore o ba file dilo ka moka tše le...   
4     ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...   

                                           tokenized  
0  [umgaqosiseko, wenza, amalungiselelo, kumaziko...  
1  [idha, iya, kuba, nobulumko, bokubeka, umseben...  
2  [he, province, of, kwazulunatal, department, o...  
3  [o, netefatša, gore, o, ba, file, dilo, ka, mo...  
4  [khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...

### Exploring the models

In [13]:
#Splitting features and target variables
X = train['text']#X is the features of the cleaned tweets
y = train['lang_id'] #Y is the target variable which is the train sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 11)

In [14]:
tfidf = TfidfVectorizer() #Call the TFidfVectorizer
cf= CountVectorizer() #Call the CountVectorizer

In [15]:
lr = LogisticRegression(C=1, class_weight='balanced', max_iter=1000)
# call the model
clf_lr = Pipeline([('tfidf', tfidf), ('clf', lr)]) #Create a pipeline
clf_lr.fit(X_train, y_train) #Fit the training data to the pipeline
y_pred_lr= clf_lr.predict(X_test)#Make predictions
print('accuracy %s' % accuracy_score(y_pred_lr, y_test)) #Print the accuracy
print('f1_score %s' % metrics.f1_score(y_test,y_pred_lr,average='weighted')) #Print the weighted f1 score
print(classification_report(y_test, y_pred_lr)) #Classification

accuracy 0.9954545454545455
f1_score 0.9954522748128464
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       323
         eng       1.00      1.00      1.00       280
         nbl       0.98      0.99      0.98       295
         nso       1.00      1.00      1.00       316
         sot       1.00      1.00      1.00       307
         ssw       0.99      1.00      0.99       295
         tsn       1.00      1.00      1.00       296
         tso       1.00      1.00      1.00       297
         ven       1.00      1.00      1.00       258
         xho       0.99      1.00      0.99       317
         zul       0.99      0.98      0.98       316

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300



In [16]:
nb = MultinomialNB()
clf_nb= Pipeline([('tfidf', tfidf), ('clf', nb)])
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred_nb, y_test)) #Print the accuracy
print('f1_score %s' % metrics.f1_score(y_test,y_pred_nb,average='weighted')) #Print the f1 score
print(classification_report(y_test, y_pred_nb)) #Print out the classification

accuracy 0.9981818181818182
f1_score 0.99818151138554
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       323
         eng       0.99      1.00      1.00       280
         nbl       0.99      1.00      0.99       295
         nso       1.00      1.00      1.00       316
         sot       1.00      1.00      1.00       307
         ssw       1.00      1.00      1.00       295
         tsn       1.00      1.00      1.00       296
         tso       1.00      1.00      1.00       297
         ven       1.00      1.00      1.00       258
         xho       1.00      1.00      1.00       317
         zul       1.00      0.99      1.00       316

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300



In [17]:
#MultinomialNB Hyperparameter tuning
tfid = TfidfVectorizer()
text = tfid.fit_transform(train['text'])
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(text,y, test_size = 0.2, random_state = 10)
params = {'alpha':[0.01,0.1,1]}

grid_MNB = GridSearchCV(MultinomialNB(), params)
grid_MNB.fit(X_train_h, y_train_h)
print(grid_MNB.best_params_)

{'alpha': 0.1}


In [20]:
multi = Pipeline([('tfid', TfidfVectorizer()),
             ('clf', MultinomialNB(alpha = 0.1))])
multi.fit(X_train, y_train)
t = test['text']
y_pred_m = multi.predict(t)
sub = pd.DataFrame( data = {'index': test['index'],
                             'lang_id': y_pred_m })
sub.to_csv('submission_m.csv', index = False)
print('accuracy %s' % accuracy_score(y_pred_nb, y_test)) #Print the accuracy
print('f1_score %s' % metrics.f1_score(y_test,y_pred_nb,average='weighted')) #Print the f1 score
print(classification_report(y_test, y_pred_nb)) #Print out the classification

accuracy 0.9981818181818182
f1_score 0.99818151138554
              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       323
         eng       0.99      1.00      1.00       280
         nbl       0.99      1.00      0.99       295
         nso       1.00      1.00      1.00       316
         sot       1.00      1.00      1.00       307
         ssw       1.00      1.00      1.00       295
         tsn       1.00      1.00      1.00       296
         tso       1.00      1.00      1.00       297
         ven       1.00      1.00      1.00       258
         xho       1.00      1.00      1.00       317
         zul       1.00      0.99      1.00       316

    accuracy                           1.00      3300
   macro avg       1.00      1.00      1.00      3300
weighted avg       1.00      1.00      1.00      3300



### Hyperparameter Tuning

In [21]:
# Make Submission
My_submission = pd.DataFrame(test['index'])
My_submission['lang_id'] = multi.predict(test['text'])
My_submission.to_csv('Nomfundo_Manyisa_Classification_Hack',index=False)